In [54]:
import requests
from massive import RESTClient
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
from datetime import date

/Users/sam/miniconda3/envs/data_science/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [67]:
class Analyze:
    """
        Makes a dictionary for each day and gives RSI, price value, and number of transactions/shares 
        traded
    """

    def __init__(self, client, day_range, ticker):
        self.client = client
        self.today = datetime.today().strftime('%Y-%m-%d')
        self.day_range = day_range
        self.ticker = ticker

    def get_business_days_back(self):
        """
        Returns the date n business days ago
        """
        current = datetime.now()
        business_days_to_sub = self.day_range
        while business_days_to_sub > 0:
            current -= timedelta(days=1)
            # If it's a weekday (Monday = 0, Sunday = 6)
            if current.weekday() < 5:  # 5 is Saturday, 6 is Sunday
                business_days_to_sub -= 1
        self.from_date = current.strftime('%Y-%m-%d')

    """
        Converts milliseconds timestamp to more readable format
    """
    def convert_timestamp(self, ts):
        return datetime.fromtimestamp(ts/1000).strftime('%Y-%m-%d')

    """
        Gets prices for a specific ticker over a set time
    """
    def get_price(self, multiplier=1, timespan='day'):
        self.time_dict = {}
        for _ in self.client.list_aggs(
                ticker=self.ticker, 
                multiplier=multiplier, 
                timespan=timespan, 
                from_=self.from_date, 
                to=self.today
        ):
            self.time_dict[self.convert_timestamp(_.timestamp)] = {
                'open': _.open, 
                'close': _.close, 
                'volume': _.volume, 
                'high': _.high, 
                'low': _.low,
                'transactions': _.transactions
            }

    """
        Gets RSI values over time frame and for a specific ticker
    """
    def get_rsi(self, timespan='day', adjusted=True, window=14, series_type='close', order='desc'):
        rsi_data = self.client.get_rsi(
            ticker=self.ticker,
            timespan=timespan,
            adjusted=adjusted,
            window=window,
            series_type=series_type,
            order=order,
            limit=self.day_range
        )
        for _ in rsi_data.values:
            timestamp = self.convert_timestamp(_.timestamp)
            if timestamp not in self.time_dict:
                print(timestamp)
            else:
                self.time_dict[timestamp]['rsi'] = _.value

    def run(self):
        self.get_business_days_back()
        self.get_price()
        self.get_rsi()
        return pd.DataFrame.from_dict(self.time_dict, orient='index')

In [ ]:
"""
    use api key and set up client
"""
api_key = 'hJQGYwLtVkOmM45yOBeEOg99YqOzJAz9'
client = RESTClient(api_key=api_key)

ticker = 'AAPL'

"""
    instance of class
"""
inst = Analyze(client=client, ticker='AAPL', day_range=21)
inst.get_business_days_back()
df = inst.run()

In [60]:
dict_data = inst.time_dict
df = pd.DataFrame.from_dict(dict_data, orient='index')

In [61]:
df

,open,close,volume,high,low,transactions,rsi
2025-10-09,257.805,254.04,38322012.0,258.000,253.1400,536585,60.855511
2025-10-10,254.940,245.27,61999098.0,256.380,244.0000,908253,47.144369
2025-10-13,249.380,247.66,38142942.0,249.690,245.5600,614923,50.422606
2025-10-14,246.600,247.77,35477986.0,248.845,244.7000,533528,50.574549
2025-10-15,249.485,249.34,33893611.0,251.820,247.4700,528880,52.798098
2025-10-16,248.250,247.45,39776974.0,249.040,245.1300,616258,49.888434
2025-10-17,248.020,252.29,49146961.0,253.380,247.2700,634830,56.499705
2025-10-20,255.885,262.24,90483029.0,264.375,255.6300,1160822,66.333258
2025-10-21,261.880,262.77,46695748.0,265.290,261.8300,714945,66.764242
2025-10-22,262.650,258.45,45011970.0,262.850,255.4300,687110,60.019782
